you should run below in the cmd for your packages installation.  
```py -m ensurepip --default-pip # remove annoying py -m```
```pip install -i https://pypi.tuna.tsinghua.edu.cn/simple mne, numpy, pandas # install the package```

In [78]:
# basic pack and function import
import numpy as np
import pandas as pd
import os
import mne

# some other basic setting
os.chdir('e:/SDODT Co., Ltd/Sleep issues')

下面是基于txt的单通道数据批处理代码。由于mne包的一些局限和代码写作原因，该脚本/函数将**仅**适用于单通道脑电设备，并可根据需要填写采样率参数以进行自动分割（默认为250，数眠设备的采样率）。暂时因为数眠设备的传导问题，对于应对短时实验任务，采用1s（默认）切片来判断伪迹会好一些。对于长时间任务（如睡眠分期），后续会制作需要手动指定切割时间（例如30s）的函数。需要注意的是，mne.make_fixed_length_epochs/events函数的特征会导致抛弃掉尾端数据（不足30s）。


还没实现的功能：  
1 多少秒1切问题。设定认为判断比例参数后如果小于参数则扔，大于则采用缩放的方式。  
2 输出不同频段或总频段的功率数据（在功率函数里面加）  
3 制定输出原始数据图、处理数据图并进行坏段标记  
4 特征提取  
5 坏采样问题判断，连续1s（250个点）内均为0认为是坏数据  

陈曦 2022.09.20 第一版

In [87]:
# 你需要做的事情：设定基础路径os.chdir('xxxx')和数据input_path路径和导出output_path路径。如有需要，在batch函数后面加入切割比例percentile，采样率s_rate和高低通滤波l_freq/h_freq。

input_path = './data/openclose_selfmotivated/raw/agg'
output_path = './data/openclose_selfmotivated/python_cleaned'

# 目前仅适用于数眠数据

def art_clean(raw, percentile = 10, s_rate = 250, l_freq = 1, h_freq = 20):  # 该函数将对text_raw对象进行预处理，并返回一个数据列表

    raw = mne.filter.filter_data(data = raw, sfreq = s_rate, l_freq = l_freq, h_freq = h_freq, verbose = 0) # 1-20Hz的FIR滤波。因为只选取了1-20Hz，50Hz的工频处理就不需要啦。另外由于是1s切片（蓝牙问题），不用特地设置数据长度（一定能被250整除）。verbose参数决定了不生成任何信息（以免太吵）。
    num_sliced = int(len(raw) / s_rate) # 保存多少秒的数据
    raw_sliced = raw.reshape(num_sliced, s_rate) # 切片

    def art_return_ss(slice_object): # 输出切片片段的平方和列表
        list_of_ss = []
        i = 0
        def sum_of_squares(lst): # 计算列表中元素的平方和
            return sum([x ** 2 for x in lst])
        for x in slice_object:
            list_of_ss.append(sum_of_squares(slice_object[i]))
            i += 1
        return(list_of_ss)

    ss = art_return_ss(raw_sliced) # 切片片段的平方和列表
    len_ss = len(ss) # 切片片段长度（几个切片）
    cut_number = int(len(ss)/percentile)

    def art_max_pos(ss, cut_number): # 需要调用art_return_ss函数并保存到某个对象ss中。此函数将返回切片对象伪迹的相对位置（排序，确定前10%）
        maxlist = sorted(ss, reverse = True)[:cut_number]  # 创建一个基于自定义数字的最大值列表
        ss_dict = {}  # 创建一个空字典
        for i, b in enumerate(ss):
            ss_dict[b] = i
        max_indices = [ss_dict[a] for a in maxlist]
        return max_indices

    def art_min_pos(ss, number): # 需要调用art_return_ss函数并保存到某个对象ss中。此函数将返回切片对象伪迹的相对位置（排序，确定后10%）
        minlist = sorted(ss, reverse = False)[:cut_number] # 创建一个基于自定义数字的最小值列表
        ss_dict = {}  # 创建一个空字典
        for i, b in enumerate(ss):
            ss_dict[b] = i
        min_indices = [ss_dict[a] for a in minlist]
        return min_indices

    art_indices = art_max_pos(ss, cut_number) + art_min_pos(ss, cut_number) # 借用上面做的函数，输出最大/小值index列表
    cleaned_sliced = [n for i, n in enumerate(raw_sliced) if i not in art_indices]
    return np.ravel(cleaned_sliced, order = 'C').tolist()

def batch(input_path, output_path, percentile = 10, s_rate = 250, l_freq = 1, h_freq = 20): # 该函数将遍历指定目录，返回filepath下全部文件名，对raw对象进行art_clean处理并返回txt对象。本函数依赖于art_clean并且只可处理数眠数据（txt格式）。
    FileName_List = os.listdir(input_path) # 返回文件名列表
    for FileName in FileName_List: # 对于所有文件名
        InputPaths = os.path.join('%s/%s' % (input_path, FileName)) # 拼接路径
        OutputPaths = os.path.join('%s/%s' % (output_path, FileName)) 
        if os.path.isfile(InputPaths): # 判断是否为文件还是文件夹。
            if InputPaths.endswith('.txt'): # 判断文件扩展名是否为指定格式
                raw = np.loadtxt(InputPaths, dtype = np.floating, delimiter = ',') # 读取文件
                result = art_clean(raw, percentile = percentile, s_rate = s_rate, l_freq = l_freq, h_freq = h_freq)
                np.savetxt(OutputPaths, result)

# 请自己加
batch(input_path, output_path)

下面是edf的原始代码，暂时抛弃（听说edf有点问题）

In [ ]:
""" from pandas.core.common import flatten # 用于解包list

# 以下基于读取edf数据
raw = mne.io.read_raw_edf(path, preload = True) # 读取文件
raw = raw.notch_filter(freqs=(50)) # 工频处理
raw = raw.filter(l_freq = 1, h_freq = 20) # 1-20Hz，FIR方法滤波
epochs = mne.make_fixed_length_epochs(raw, duration = 1) # 切片
epochs_data = epochs.get_data()

def art_return_ss(raw, dur = 1): # 返回raw对象的切片平方和
    import mne
    from pandas.core.common import flatten
    epochs = mne.make_fixed_length_epochs(raw, duration = dur) # 切片，这块需要重写！！！！
    epochs_data = epochs.get_data()
    list_of_ss = []
    i = 0
    def sum_of_squares(lst): # 计算列表中元素的平方和
        return sum([x ** 2 for x in lst])
    for x in epochs_data: # 输出切片片段的平方和列表
        list_of_ss.append(sum_of_squares(list(flatten(epochs_data[i]))))
        i += 1
    return list_of_ss

ss = art_return_ss(raw, 1) # 切片平方和原始数据


# 这段代码有优化的空间，可以使用heapq.nlargest/nsmallest(n, list)函数，据说会更快，没试过
# 该代码将对raw对象进行切片处理（根据自定义dur时长），随后对切片平方和原始数据进行排序，取number数量清除前后多少个值，随后根据相对位置清除epochs data，并将所有数据重新连接在一起（应该会自动连接）
def art_max_pos(ss, number): # 需要调用art_return_ss函数并保存到某个对象ss中。此函数将返回切片对象伪迹的相对位置（排序，确定前10%）
    maxlist = sorted(ss, reverse = True)[:number]  # 创建一个基于自定义数字的最大值列表
    ss_dict = {}  # 创建一个空字典
    for i, b in enumerate(ss):
        ss_dict[b] = i
    max_indices = [ss_dict[a] for a in maxlist]
    return max_indices

def art_min_pos(ss, number): # 需要调用art_return_ss函数并保存到某个对象ss中。此函数将返回切片对象伪迹的相对位置（排序，确定后10%）
    minlist = sorted(ss, reverse = False)[:number] # 创建一个基于自定义数字的最小值列表
    ss_dict = {}  # 创建一个空字典
    for i, b in enumerate(ss):
        ss_dict[b] = i
    min_indices = [ss_dict[a] for a in minlist]
    return min_indices

def art_clean(raw, number, dur = 1): 
    
    import mne
    from pandas.core.common import flatten
    epochs = mne.make_fixed_length_epochs(raw, duration = dur) # 切片，这块可能需要重写
    epochs_data = epochs.get_data()

    def art_return_ss(epochs_df): # 返回raw对象的切片平方和
        list_of_ss = []
        i = 0
        def sum_of_squares(lst): # 计算列表中元素的平方和
            return sum([x ** 2 for x in lst])
        for x in epochs_data: # 输出切片片段的平方和列表
            list_of_ss.append(sum_of_squares(list(flatten(epochs_df[i]))))
            i += 1
        return list_of_ss
    
    ss = art_return_ss(epochs_data) 

    def art_max_pos(ss, number): # 需要调用art_return_ss函数并保存到某个对象ss中。此函数将返回切片对象伪迹的相对位置（排序，确定前10%）
        maxlist = sorted(ss, reverse = True)[:number]  # 创建一个基于自定义数字的最大值列表
        ss_dict = {}  # 创建一个空字典
        for i, b in enumerate(ss):
            ss_dict[b] = i
        max_indices = [ss_dict[a] for a in maxlist]
        return max_indices
    
    def art_min_pos(ss, number): # 需要调用art_return_ss函数并保存到某个对象ss中。此函数将返回切片对象伪迹的相对位置（排序，确定后10%）
        minlist = sorted(ss, reverse = False)[:number] # 创建一个基于自定义数字的最小值列表
        ss_dict = {}  # 创建一个空字典
        for i, b in enumerate(ss):
            ss_dict[b] = i
        min_indices = [ss_dict[a] for a in minlist]
        return min_indices
    
    max_indices = art_max_pos(ss, number) # 借用上面做的函数，输出最大值列表
    min_indices = art_min_pos(ss, number) # 借用上面做的函数，输出最小值列表

epochs = mne.make_fixed_length_epochs(raw, duration = 1) # 切片
epochs_data = epochs.get_data()

max_indices = art_max_pos(ss = ss, number = 10)
min_indices = art_min_pos(ss = ss, number = 10)

# print(raw)
# print(109/3)
# print(raw.get_data()) # = print(raw[0][0])
# print(raw[0])
# print(raw[0][1])
# print(epochs_data)
# print(raw[0][0])
# print(raw[0][0][0]) # 要删的是这个
# print(len(raw[0][0][0])) # 原始数据列表 27250个点
# print(raw[0][0][0][0]) """